# Python on-the-fly colormap generator

Useful bit of code that allows you to see how python plots look on-the-fly.

In [ ]:
import Make_EQ as eq
from ipywidgets import interactive, VBox, Layout, widgets, fixed
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import os
import xarray as xr
import matplotlib

In [ ]:
log_var = False
bbox = [5, 20, -35, -15]
reduce = 5
example = 'CHL'
if example == 'CHL':
    # CHL EXAMPLE
    varfile = '/home/jovyan/work/products/dataset-oc-glo-chl-multi_cci-l4-chl_4km_monthly-rep-v02_1568876553803.nc'
    varname = 'CHL'
    lonname = 'longitude'
    latname = 'latitude'
    log_var = True
    # Viridis
    nchan = 12
    channel_red = [] ; channel_green = [] ; channel_blue = []
    for ii in np.arange(nchan):
        channel_red.append(plt.cm.viridis(ii/(nchan-1))[0])
        channel_green.append(plt.cm.viridis(ii/(nchan-1))[1])
        channel_blue.append(plt.cm.viridis(ii/(nchan-1))[2])
elif example == 'SST':
    # SST EXAMPLE
    varfile = '/home/jovyan/work/products/METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2_1568885414480.nc'
    varname = 'analysed_sst'
    lonname = 'lon'
    latname = 'lat'
    # Medspiration
    channel_red=[1,1,0,0,0,0,0,0.5,1,1,1,0.25]
    channel_green=[1,0.4,0,0.5,1,0.7,0.42,0.7,1,0.5,0,0]
    channel_blue=[1,1,0.2,0.6,1,0.5,0,0,0,0,0,0]


In [ ]:
# open data
ds1 = xr.open_dataset(varfile)
LAT = ds1.variables[latname].values
LON = ds1.variables[lonname].values
VAR = ds1.variables[varname].values
ds1.close()

# process the data a bit (subset and get max/min)
VAR, LON, LAT, vlimits = eq.process_data(VAR, LON, LAT, bbox, varname, reduce, log_var=log_var)

In [ ]:
%matplotlib inline

nchan = len(channel_red)
print('nChannels: '+str(nchan))

# build the widgets
red_widgets, green_widgets, blue_widgets = eq.make_widgets(channel_red, channel_green, channel_blue)

# build the plot command
runCMD = eq.build_plot_command(vlimits,\
                               red_widgets, green_widgets, blue_widgets)
# run the plot command
exec(runCMD)
box_layout = Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='center')

# display the plot
for ii in range(0, nchan):
    display(VBox([iplot.children[ii], iplot.children[ii+nchan],iplot.children[ii+nchan*2]],\
                  layout=box_layout))
display(iplot.children[-1])

In [ ]:
# interpolate colour map onto required range / steps
I_am_done = False

if I_am_done:
    reds = interactive_plot.result[2]
    greens = interactive_plot.result[3]
    blues = interactive_plot.result[4]
    tscale_orig = np.linspace(vmin, vmax, len(reds))
    tscale_new = np.linspace(vmin, vmax, vsteps)

    interp_redf = interpolate.interp1d(tscale_orig, reds)
    interp_greenf = interpolate.interp1d(tscale_orig, greens)
    interp_bluef = interpolate.interp1d(tscale_orig, blues)

    interp_red = np.array(interp_redf(tscale_new)*255.).astype(int)
    interp_green = np.array(interp_greenf(tscale_new)*255.).astype(int)
    interp_blue = np.array(interp_bluef(tscale_new)*255.).astype(int)

In [ ]:
if I_am_done:
    # write to file
    my_file = 'Test_colbar.grass'
    if os.path.exists(my_file):
        os.remove(my_file)

    with open(my_file, 'w') as the_file:
        for ii in range(0,len(interp_red)):
            the_file.write(str(tscale_new[ii]) + ' ' + \
                           str(interp_red[ii]) + ' ' + \
                           str(interp_green[ii]) + ' ' + \
                           str(interp_blue[ii]))
            if ii != len(interp_red)-1:
                the_file.write('\n')